# TCGA-UCEC – Notebook 02
## Harmonisation des échantillons

Objectif : harmoniser les identifiants d échantillons entre les données RNA-seq et cliniques, vérifier la cohérence et préparer un jeu de données propre pour la suite du pipeline.

## 1. Imports et configuration

In [1]:
# ==========================================================================================================
import os                                       # Navigation fichiers (DIRS, chemins relatifs)
import warnings                                 # Masquer warnings (dépréciation)
import gc                                       # Gestion mémoire (nettoyage objets inutilisés)
import json                                     # Lecture du dictionnaire de métadonnées

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

# ==========================================================================================================
PROJECT_ROOT = r"C:\Z\M2_AIDA\TCGA_UCEC_project" #Laïla

DIRS = {
    # Racine data
    "DATA": os.path.join(PROJECT_ROOT, "data"),

    # Données
    "RAW": os.path.join(PROJECT_ROOT, "data", "raw"),

    # Données intermédiaires (persistées)
    "PROCESSED":        os.path.join(PROJECT_ROOT, "data", "processed"),
    "NORM":             os.path.join(PROJECT_ROOT, "data", "processed", "normalized"),
    "QC_FILTERED":      os.path.join(PROJECT_ROOT, "data", "processed", "qc_filtered"),
    "COHORT_FILTERED":  os.path.join(PROJECT_ROOT, "data", "processed", "cohort_filtered"),
    "SC_OBJECTS":       os.path.join(PROJECT_ROOT, "data", "processed", "single_cell_objects"),
    "PSEUDOBULK_PROC":  os.path.join(PROJECT_ROOT, "data", "processed", "pseudobulk"),

    # Artefacts (par étape du pipeline)
    "ARTEFACTS": os.path.join(PROJECT_ROOT, "data", "artefacts"),

    "EDA":         os.path.join(PROJECT_ROOT, "data", "artefacts", "exploratory_data_analysis"),
    "QC":          os.path.join(PROJECT_ROOT, "data", "artefacts", "qc_analysis"),
    "COHORT":      os.path.join(PROJECT_ROOT, "data", "artefacts", "cohort_selection"),
    "SINGLE_CELL": os.path.join(PROJECT_ROOT, "data", "artefacts", "single_cell_analysis"),
    "PSEUDOBULK":  os.path.join(PROJECT_ROOT, "data", "artefacts", "pseudobulk_preparation"),
    "DE":          os.path.join(PROJECT_ROOT, "data", "artefacts", "differential_expression"),
    "ENRICH":      os.path.join(PROJECT_ROOT, "data", "artefacts", "functional_enrichment"),

    # Autres dossiers du projet
    "RESULTS_R": os.path.join(PROJECT_ROOT, "Results_R_Analysis"),
    "TMP":       os.path.join(PROJECT_ROOT, "tmp_cache"),
    "DOCS":      os.path.join(PROJECT_ROOT, "documentation"),
}
for path in DIRS.values():
    os.makedirs(path, exist_ok=True)
os.chdir(PROJECT_ROOT)

# ==========================================================================================================
warnings.filterwarnings("ignore") 
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.figsize'] = (10, 6) 

# ==========================================================================================================
print(f"✅ Environnement chargé. Working directory: {os.getcwd()}")


✅ Environnement chargé. Working directory: c:\Z\M2_AIDA\TCGA_UCEC_project


## 2. Chargement des données

In [2]:
expr_filename = "TCGA-UCEC.star_counts.tsv.gz"
clin_filename = "TCGA-UCEC.clinical.tsv.gz"

expr_path = os.path.join(DIRS["RAW"], expr_filename)
clin_path = os.path.join(DIRS["RAW"], clin_filename)

expr = pd.read_csv(expr_path, sep='\t')
clin = pd.read_csv(clin_path, sep='\t')

# ==========================================================================================================
print(
    f"✅ Datasets successfully loaded\n"
    f"   - Expression matrix: {expr_path} | shape (genes × samples): {expr.shape}\n"
    f"   - Clinical matrix:   {clin_path} | shape (samples × variables): {clin.shape}"
)

✅ Datasets successfully loaded
   - Expression matrix: C:\Z\M2_AIDA\TCGA_UCEC_project\data\raw\TCGA-UCEC.star_counts.tsv.gz | shape (genes × samples): (60660, 586)
   - Clinical matrix:   C:\Z\M2_AIDA\TCGA_UCEC_project\data\raw\TCGA-UCEC.clinical.tsv.gz | shape (samples × variables): (600, 79)


## 3. Vérification des identifiants échantillons

In [3]:
expr_samples = set(expr.columns)
clin_samples = set(clin.index)

print(f'Nombre d échantillons RNA-seq : {len(expr_samples)}')
print(f'Nombre d échantillons cliniques : {len(clin_samples)}')

Nombre d échantillons RNA-seq : 586
Nombre d échantillons cliniques : 600


## 4. Intersection des échantillons

In [4]:
common_samples = sorted(expr_samples.intersection(clin_samples))
print(f'Nombre d échantillons communs : {len(common_samples)}')

Nombre d échantillons communs : 0


**Problème d'harmonisation détecté: Nombre d'échantillons communs = 0**

Les identifiants des données RNA-seq et cliniques sont fournis à des niveaux différents (échantillon vs patient). Une harmonisation préalable des barcodes TCGA au niveau patient est donc nécessaire avant toute intersection.


In [5]:
if "submitter_id" in clin.columns:
    clin_patient_col = "submitter_id"
    print("Identifiant patient clinique utilise : submitter_id")
elif "case_submitter_id" in clin.columns:
    clin_patient_col = "case_submitter_id"
    print("Identifiant patient clinique utilise : case_submitter_id")
else:
    clin_patient_col = None
    print("Aucun identifiant patient standard trouve (submitter_id / case_submitter_id)")


Identifiant patient clinique utilise : submitter_id


In [6]:
print("=== Mise en evidence du probleme d'harmonisation des identifiants ===\n")

# Exemples cote expression
print("Exemples d'identifiants cote expression (colonnes) :")
for x in list(expr.columns[:5]):
    print(f"  - {x}")

print("\nExemples d'identifiants cote clinique (submitter_id) :")
for x in clin["submitter_id"].head().tolist():
    print(f"  - {x}")

print("\nInterpretation :")
print(
    "- Les donnees RNA-seq sont indexees par identifiants patient au niveau colonne\n"
    "- Les donnees cliniques contiennent les identifiants patient dans une colonne\n"
    "- Les deux sources utilisent le meme identifiant logique (patient TCGA)\n"
    "- MAIS elles ne sont pas structurees de la meme maniere\n"
)

print(
    "Conclusion :\n"
    "Une intersection directe est invalide.\n"
    "Une harmonisation structurelle est necessaire avant toute analyse conjointe.\n"
)


=== Mise en evidence du probleme d'harmonisation des identifiants ===

Exemples d'identifiants cote expression (colonnes) :
  - Ensembl_ID
  - TCGA-FI-A3PX-01A
  - TCGA-BG-A221-01A
  - TCGA-EY-A1GK-01A
  - TCGA-BG-A2AE-01A

Exemples d'identifiants cote clinique (submitter_id) :
  - TCGA-AJ-A3NC
  - TCGA-AJ-A3NC
  - TCGA-KP-A3W0
  - TCGA-B5-A11N
  - TCGA-BS-A0V7

Interpretation :
- Les donnees RNA-seq sont indexees par identifiants patient au niveau colonne
- Les donnees cliniques contiennent les identifiants patient dans une colonne
- Les deux sources utilisent le meme identifiant logique (patient TCGA)
- MAIS elles ne sont pas structurees de la meme maniere

Conclusion :
Une intersection directe est invalide.
Une harmonisation structurelle est necessaire avant toute analyse conjointe.



In [7]:
expr_patient_ids = {
    c[:12] for c in expr.columns
    if isinstance(c, str) and c.startswith("TCGA-")
}

clin_patient_ids = set(
    clin[clin_patient_col].dropna().astype(str).str.slice(0, 12)
)

common_patients = sorted(expr_patient_ids.intersection(clin_patient_ids))

print(f"Patients expression (unique) : {len(expr_patient_ids)}")
print(f"Patients clinique (unique)   : {len(clin_patient_ids)}")
print(f"Patients communs             : {len(common_patients)}")
print("Exemples patients communs :", common_patients[:5])


Patients expression (unique) : 557
Patients clinique (unique)   : 560
Patients communs             : 557
Exemples patients communs : ['TCGA-2E-A9G8', 'TCGA-4E-A92E', 'TCGA-5B-A90C', 'TCGA-5S-A9Q8', 'TCGA-A5-A0G1']


**Remarque sur le second rechargement des données d'expression**

Les données RNA-seq ont été rechargées à ce stade avec le paramètre `index_col=0` afin de corriger la structure initiale de la matrice d’expression.

Lors du premier chargement, la colonne `Ensembl_ID` était interprétée comme une colonne standard, ce qui mélangeait identifiants de gènes et identifiants d’échantillons dans la matrice. Cette structure incorrecte empêchait toute harmonisation fiable des identifiants TCGA entre les données transcriptomiques et cliniques.

Le rechargement explicite avec `index_col=0` permet de définir correctement les identifiants de gènes comme index de la matrice, et de conserver uniquement les échantillons TCGA dans les colonnes. Cette correction structurelle est nécessaire avant toute harmonisation au niveau patient et avant toute analyse conjointe expression–clinique.


## 5. Filtrage des données sur les échantillons communs

In [8]:
# --- 5.1 Reindexation de la table clinique au niveau patient
clin_filtered = (
    clin
    .set_index(clin_patient_col)
    .loc[common_patients]
)

# --- 5.2 Filtrage de la matrice d'expression
# On conserve les colonnes dont le patient (12 premiers caracteres) est dans common_patients
expr_filtered = expr.loc[
    :,
    [col for col in expr.columns if col[:12] in common_patients]
]

# --- 5.3 Verifications finales
assert expr_filtered.shape[1] > 0
assert clin_filtered.shape[0] > 0
assert set(common_patients).issubset(set(clin_filtered.index))
print(
    f"Expression filtree : {expr_filtered.shape} (genes x echantillons)\n"
    f"Clinique filtree   : {clin_filtered.shape} (patients x variables)"
)
print("✅ Filtrage valide : expression et clinique correctement alignees au niveau patient")

assert set(common_patients).issubset(set(clin_filtered.index))
print('✅ Harmonisation des échantillons validée')


Expression filtree : (60660, 585) (genes x echantillons)
Clinique filtree   : (597, 78) (patients x variables)
✅ Filtrage valide : expression et clinique correctement alignees au niveau patient
✅ Harmonisation des échantillons validée


## 6. Prochaines étapes
- QC échantillons (outliers globaux)
- Identification et sélection des labels de sous-types TCGA
- QC des gènes (faible expression)